<a href="https://colab.research.google.com/github/Sohag016/CodeAlpha_HandwrittenCharacterRecognition/blob/main/Hand_Written_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-datasets

In [2]:
import os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report, confusion_matrix

print("TensorFlow:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def plot_history(history, title="Training"):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history.get('val_accuracy', []), label='val_acc')
    plt.title(f"{title} — Accuracy"); plt.xlabel("Epoch"); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history.get('val_loss', []), label='val_loss')
    plt.title(f"{title} — Loss"); plt.xlabel("Epoch"); plt.legend()
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ','_')}.png")
    plt.close()

In [4]:
def show_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title); plt.xlabel("Predicted"); plt.ylabel("True")
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ','_')}.png")
    plt.close()

In [5]:
def show_samples(ds, preprocess_fn, n=6, title="Samples"):
    ds_sh = ds.map(preprocess_fn)
    plt.figure(figsize=(12,2))
    for i, (img, lbl) in enumerate(ds_sh.take(n)):
        plt.subplot(1,n,i+1)
        plt.imshow(tf.squeeze(img), cmap='gray')
        plt.title(int(lbl.numpy()))
        plt.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ','_')}.png")
    plt.close()

In [6]:

# MNIST quick model

print("\n<<<<<<<<=== MNIST quick model ===>>>>>>>>>>")
(ds_mnist_train, ds_mnist_test), mnist_info = tfds.load('mnist', split=['train','test'], as_supervised=True, with_info=True)

IMG = 28
AUTOTUNE = tf.data.AUTOTUNE
MNIST_BATCH = 256

def preprocess_mnist(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [IMG,IMG,1])
    return image, label

mnist_train = ds_mnist_train.map(preprocess_mnist, num_parallel_calls=AUTOTUNE).shuffle(10000).batch(MNIST_BATCH).prefetch(AUTOTUNE)
mnist_test  = ds_mnist_test.map(preprocess_mnist, num_parallel_calls=AUTOTUNE).batch(MNIST_BATCH).prefetch(AUTOTUNE)

show_samples(ds_mnist_train, preprocess_mnist, n=6, title="MNIST_samples")

def build_mnist_model():
    inp = tf.keras.Input((IMG,IMG,1))
    x = tf.keras.layers.Conv2D(32,3,padding='same',activation='relu')(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64,3,padding='same',activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    out = tf.keras.layers.Dense(10,activation='softmax')(x)
    return tf.keras.Model(inp,out)

mnist_model = build_mnist_model()
mnist_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
mnist_model.summary()


<<<<<<<<=== MNIST quick model ===>>>>>>>>>>


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,994 (6.20 MB)

 Trainable params: 1,625,930 (6.20 MB)

 Non-trainable params: 64 (256.00 B)

In [7]:
mnist_ckpt = "mnist_best.keras"
mnist_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(mnist_ckpt, monitor='val_accuracy', save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True, verbose=1)
]

In [8]:
print("Training MNIST (this should be fast on GPU)...")
hist_mnist = mnist_model.fit(mnist_train, validation_data=mnist_test, epochs=30, callbacks=mnist_callbacks, verbose=2)
plot_history(hist_mnist, "MNIST_Training")

mnist_eval = mnist_model.evaluate(mnist_test, verbose=2)
print(f"MNIST test accuracy: {mnist_eval[1]*100:.2f}%")

Training MNIST (this should be fast on GPU)...
Epoch 1/30

Epoch 1: val_accuracy improved from -inf to 0.97510, saving model to mnist_best.keras
235/235 - 18s - 76ms/step - accuracy: 0.8687 - loss: 0.4278 - val_accuracy: 0.9751 - val_loss: 1.1479 - learning_rate: 1.0000e-03
Epoch 2/30

Epoch 2: val_accuracy improved from 0.97510 to 0.98610, saving model to mnist_best.keras
235/235 - 4s - 16ms/step - accuracy: 0.9535 - loss: 0.1525 - val_accuracy: 0.9861 - val_loss: 0.1442 - learning_rate: 1.0000e-03
Epoch 3/30

Epoch 3: val_accuracy improved from 0.98610 to 0.98660, saving model to mnist_best.keras
235/235 - 4s - 19ms/step - accuracy: 0.9653 - loss: 0.1142 - val_accuracy: 0.9866 - val_loss: 0.0432 - learning_rate: 1.0000e-03
Epoch 4/30

Epoch 4: val_accuracy improved from 0.98660 to 0.98920, saving model to mnist_best.keras
235/235 - 4s - 16ms/step - accuracy: 0.9700 - loss: 0.0975 - val_accuracy: 0.9892 - val_loss: 0.0343 - learning_rate: 1.0000e-03
Epoch 5/30

Epoch 5: val_accuracy d

In [9]:
y_true_m = np.concatenate([y.numpy() for _, y in mnist_test], axis=0)
y_pred_m = np.argmax(mnist_model.predict(mnist_test), axis=1)
print("MNIST classification report:")
print(classification_report(y_true_m, y_pred_m, digits=4))
show_confusion_matrix(y_true_m, y_pred_m, title="MNIST_Confusion_Matrix")

mnist_model.save("mnist_model_final.keras")
print("Saved mnist_model_final.keras")

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
MNIST classification report:
              precision    recall  f1-score   support

           0     0.9898    0.9949    0.9924       980
           1     0.9947    0.9991    0.9969      1135
           2     0.9904    0.9961    0.9932      1032
           3     0.9921    0.9960    0.9941      1010
           4     0.9939    0.9929    0.9934       982
           5     0.9888    0.9888    0.9888       892
           6     0.9948    0.9916    0.9932       958
           7     0.9951    0.9883    0.9917      1028
           8     0.9938    0.9887    0.9913       974
           9     0.9900    0.9861    0.9881      1009

    accuracy                         0.9924     10000
   macro avg     0.9923    0.9923    0.9923     10000
weighted avg     0.9924    0.9924    0.9924     10000

Saved mnist_model_final.keras


In [10]:

# EMNIST advanced pipeline

print("\n\n<<<<<<<<<<=== EMNIST advanced pipeline ===>>>>>>>>>>>>")
(ds_emnist_train, ds_emnist_test), emnist_info = tfds.load('emnist/balanced', split=['train','test'], as_supervised=True, with_info=True)
print("EMNIST classes:", emnist_info.features['label'].num_classes)

def preprocess_emnist(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.transpose(image, perm=[1,0,2])
    image = tf.image.flip_left_right(image)
    image = tf.reshape(image, [IMG,IMG,1])
    return image, label

show_samples(ds_emnist_train, preprocess_emnist, n=6, title="EMNIST_samples")

EMNIST_BATCH = 256
emnist_train = ds_emnist_train.map(preprocess_emnist, num_parallel_calls=AUTOTUNE).shuffle(100000).batch(EMNIST_BATCH).prefetch(AUTOTUNE)
emnist_test  = ds_emnist_test.map(preprocess_emnist, num_parallel_calls=AUTOTUNE).batch(EMNIST_BATCH).prefetch(AUTOTUNE)

# MixUp functions
def mixup_tf(images, labels_onehot, alpha=0.2):
    B = tf.shape(images)[0]
    lam = tf.random.uniform([], 0.0, 1.0)
    lam = tf.maximum(lam, 1.0 - lam)
    idx = tf.random.shuffle(tf.range(B))
    mixed_images = lam * images + (1.0 - lam) * tf.gather(images, idx)
    mixed_labels = lam * labels_onehot + (1.0 - lam) * tf.gather(labels_onehot, idx)
    return mixed_images, mixed_labels

def prepare_emnist_for_mixup(ds, batch_size):
    def to_onehot(images, labels):
        num_classes = emnist_info.features['label'].num_classes
        labels_oh = tf.one_hot(labels, depth=num_classes)
        return images, labels_oh
    ds = ds.map(to_onehot, num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x,y: (x, tf.cast(y, tf.float32)), num_parallel_calls=AUTOTUNE)
    return ds

emnist_train_for_mix = prepare_emnist_for_mixup(emnist_train, EMNIST_BATCH)
emnist_train_mixed = emnist_train_for_mix.map(lambda x,y: mixup_tf(x,y,0.2), num_parallel_calls=AUTOTUNE)




<<<<<<<<<<=== EMNIST advanced pipeline ===>>>>>>>>>>>>


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.L21HHS_3.1.0/emnist-train.tfrecord*...:   0%|  …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/balanced/incomplete.L21HHS_3.1.0/emnist-test.tfrecord*...:   0%|   …

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/balanced/3.1.0. Subsequent calls will reuse this data.
EMNIST classes: 47


In [11]:
# EMNIST model
from tensorflow.keras import regularizers

def build_emnist_model(num_classes):
    inp = tf.keras.Input((IMG,IMG,1))
    aug = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(0.12),
        tf.keras.layers.RandomTranslation(0.12,0.12),
        tf.keras.layers.RandomZoom(0.12),
        tf.keras.layers.RandomContrast(0.12),
    ])
    x = aug(inp)
    for filters in [64,128,256,512]:
        x = tf.keras.layers.Conv2D(filters,3,padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.Conv2D(filters,3,padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.MaxPooling2D((2,2))(x)
        x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    out = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inp,out)

emnist_num_classes = emnist_info.features['label'].num_classes
emnist_model = build_emnist_model(emnist_num_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05)
emnist_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
emnist_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,978,671 (18.99 MB)

 Trainable params: 4,974,831 (18.98 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [12]:
# Callbacks
emnist_ckpt = "emnist_best.keras"
emnist_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(emnist_ckpt, monitor='val_accuracy', save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True, verbose=1)
]


In [13]:
def to_onehot_ds(ds):
    def _onehot(img, lbl):
        lbl_oh = tf.one_hot(lbl, depth=emnist_num_classes)
        return img, tf.cast(lbl_oh, tf.float32)
    return ds.map(_onehot, num_parallel_calls=AUTOTUNE)

emnist_val_ds = to_onehot_ds(emnist_test)

In [14]:
print("\nTraining EMNIST advanced model (mixup + augmentation).")
hist_emnist = emnist_model.fit(emnist_train_mixed,
                               validation_data=emnist_val_ds,
                               epochs=80,
                               callbacks=emnist_callbacks,
                               verbose=2)

plot_history(hist_emnist, title="EMNIST_Training")


Training EMNIST advanced model (mixup + augmentation).
Epoch 1/80

Epoch 1: val_accuracy improved from -inf to 0.53362, saving model to emnist_best.keras
441/441 - 70s - 159ms/step - accuracy: 0.3209 - loss: 3.2507 - val_accuracy: 0.5336 - val_loss: 2.1301 - learning_rate: 1.0000e-03
Epoch 2/80

Epoch 2: val_accuracy improved from 0.53362 to 0.82330, saving model to emnist_best.keras
441/441 - 48s - 108ms/step - accuracy: 0.6435 - loss: 2.4610 - val_accuracy: 0.8233 - val_loss: 1.1914 - learning_rate: 1.0000e-03
Epoch 3/80

Epoch 3: val_accuracy improved from 0.82330 to 0.83612, saving model to emnist_best.keras
441/441 - 48s - 109ms/step - accuracy: 0.6854 - loss: 2.2883 - val_accuracy: 0.8361 - val_loss: 1.1640 - learning_rate: 1.0000e-03
Epoch 4/80

Epoch 4: val_accuracy improved from 0.83612 to 0.84787, saving model to emnist_best.keras
441/441 - 48s - 109ms/step - accuracy: 0.6936 - loss: 2.2167 - val_accuracy: 0.8479 - val_loss: 1.0981 - learning_rate: 1.0000e-03
Epoch 5/80

Epo

In [15]:
# Evaluate EMNIST
y_true_e = np.concatenate([y.numpy() for _, y in emnist_test], axis=0)
probs = emnist_model.predict(emnist_test, verbose=1)
y_pred_e = np.argmax(probs, axis=1)

print(f"\nEMNIST test accuracy (final): {np.mean(y_true_e == y_pred_e)*100:.2f}%")
print("\nEMNIST Classification report:")
print(classification_report(y_true_e, y_pred_e, digits=4))
show_confusion_matrix(y_true_e, y_pred_e, title="EMNIST_Confusion_Matrix")

74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step

EMNIST test accuracy (final): 89.02%

EMNIST Classification report:
              precision    recall  f1-score   support

           0     0.6329    0.8275    0.7172       400
           1     0.4512    0.9125    0.6038       400
           2     0.9372    0.8950    0.9156       400
           3     0.9851    0.9925    0.9888       400
           4     0.9621    0.9525    0.9573       400
           5     0.9464    0.9275    0.9369       400
           6     0.9426    0.9450    0.9438       400
           7     0.9684    0.9950    0.9815       400
           8     0.9772    0.9625    0.9698       400
           9     0.6176    0.8925    0.7301       400
          10     0.9777    0.9850    0.9813       400
          11     0.9727    0.9800    0.9763       400
          12     0.9695    0.9550    0.9622       400
          13     0.9227    0.9550    0.9386       400
          14     0.9850    0.9875    0.9863       400
          15     0.8065   

In [18]:
# Print final accuracies
mn_acc = mnist_eval[1]*100
em_acc = np.mean(y_true_e == y_pred_e)*100
print(f"MNIST accuracy: {mn_acc:.2f}%")
print(f"EMNIST accuracy: {em_acc:.2f}%")

MNIST accuracy: 99.24%
EMNIST accuracy: 89.02%
